In [ ]:
# Calculating sub-metric c) for thrashing, N=3 in this script.
# c) Repeated modifications to the same region of code within N successive commits.
# looked at the line number ranges in three successive commits (commit n, commit n+1, and commit n+2). 
# If there is 50% overlap or more in these ranges, we considered this to be repeated modifications to the same region of code. 
# Each case would be a specific region of the code in a specific file.

# function: 50% overlap criterion checking
def overlap_percent_check(list1,list2,list3):
    count_repeat_modified_area = False
    start_lines = []
    end_lines = []
    modified_lines = []
    #['', '347', '7', '350', '21']
    three_hunks = [list1,list2,list3]
    for n, each_h in enumerate(three_hunks):
        # current hunk header
        hunk_header = ''
        for n, m in enumerate(each_h):
            if m.startswith('@@'):
                hunk_header = m
        for n, m in enumerate(each_h):
            s = ''
            s_list = []
            if hunk_header != '' and hunk_header != ' ':
                s = hunk_header.split('@@')[1]
                s_list = re.split('[-,+]',s)
                start_line = 0
                end_line = 0
                total_modified_line = 0
                if len(s_list) == 5:
                    # default value
                    start_line = int(s_list[3])
                    total_modified_line = int(s_list[4])
                    # the start code line is the same
                    if int(s_list[1]) == int(s_list[3]):
                        if int(s_list[2]) > int(s_list[4]):
                            total_modified_line = int(s_list[2])
                    # the start code line is not the same                         
                    if int(s_list[1]) != int(s_list[3]):
                        if int(s_list[1]) < int(s_list[3]):
                            start_line = int(s_list[1])
                        a = int(s_list[1]) + int(s_list[2])
                        b = int(s_list[3]) + int(s_list[4])
                        if a > b:
                            total_modified_line = a - start_line
                        else:
                            total_modified_line = b - start_line
                    end_line = start_line + total_modified_line            
        start_lines.append(start_line)
        end_lines.append(end_line)
        modified_lines.append(total_modified_line)
            
    # find out the overlapping changed code lines
    common_changed_start = max(start_lines)  
    common_changed_end = min(end_lines)
    max_of_overall_region = max(modified_lines)
    total_common_lines = int(common_changed_end) - int(common_changed_start)
    #  the 50% overlap criterion   
    if int(total_common_lines) > 0 and (total_common_lines/max_of_overall_region) >= 0.5:
        count_repeat_modified_area = True           
    return count_repeat_modified_area    
    
# function: for the same file, handling different hunk numbers
def handle_different_number_of_hunks(list1,list2,list3,count_hunks_first,count_hunks_second,count_hunks_third):
    count_repeat_change = 0
    # for check hunk numbers
    hunk_distribu = [len(count_hunks_first),len(count_hunks_second),len(count_hunks_third)]
       
    # for the same modified file x, excluding cases where a significant number of lines were deleted in file X between (for instance) Commit 1 and Commit 2.
    # @@-56,76+56,4@@  ->  #['', '347', '7', '350', '21']
    first_two_commit = [list1,list2]
    for n, each_commit in enumerate(first_two_commit):
        for i, v in enumerate(each_commit):
            if v.startswith('@@'):
                s = v.split('@@')[1]
                s_list = re.split('[-,+]',s)
                if len(s_list) == 5:
                    if int(s_list[2]) and int(s_list[4]) and (int(s_list[2]) - int(s_list[4])) > 60:
                        count_repeat_change = 0
                        return count_repeat_change        
    # 1.1 hunk numbers [1,1,1] - all had changed only one code area
    if hunk_distribu.count(1) == 3:
        if overlap_percent_check(list1,list2,list3) == True:
            count_repeat_change += 1 

    # 1.2 [1，1，>1] - one commit had changed more than one code areas, the other two both had changed only one code area 
    if hunk_distribu.count(1) == 2:
        iter_commit = []
        iter_hunk_list = []
        if len(count_hunks_first) >1:
            iter_hunk_list = count_hunks_first
            iter_commit = list1
        if len(count_hunks_second) >1:
            iter_hunk_list = count_hunks_second
            iter_commit = list2
        if len(count_hunks_third) >1:
            iter_hunk_list = count_hunks_third
            iter_commit = list3
        different_hunk_segment = []
        for i in range(len(iter_hunk_list)):
            each_hunk_list = ''
            if 0 <= i < len(iter_hunk_list)-1:
                each_hunk_list =  iter_commit[iter_hunk_list[i]:iter_hunk_list[i+1]]        
            if i == len(iter_hunk_list)-1:
                each_hunk_list =  iter_commit[iter_hunk_list[i]:]
            different_hunk_segment.append(each_hunk_list)
                    
        for n, m in enumerate(different_hunk_segment):
            c1 = False
            # each hunk starts with @@
            if iter_commit == list1:
                c1 = overlap_percent_check(m,list2,list3)
            if iter_commit == list2:
                c1 = overlap_percent_check(list1,m,list3)
            if iter_commit == list3:
                c1 = overlap_percent_check(list1,list2,m)
            if c1 == True:
                count_repeat_change += 1 
            c1 = False

    # 1.3 [1，>1,>1] - two commit had changed more than one code areas, the other one had changed only one code area 
    if hunk_distribu.count(1) == 1:
        target_commit = []
        iter_commit1 = []
        iter_commit2 = []
        iter_hunk1 = []
        iter_hunk2 = []
        if len(count_hunks_first) == 1:
            target_commit = list1
            iter_hunk1 = count_hunks_second
            iter_hunk2 = count_hunks_third
            iter_commit1 = list2
            iter_commit2 = list3
        if len(count_hunks_second) == 1:
            target_commit = list2
            iter_hunk1 = count_hunks_first
            iter_hunk2 = count_hunks_third
            iter_commit1 = list1
            iter_commit2 = list3
        if len(count_hunks_third) == 1:
            target_commit = list3
            iter_hunk1 = count_hunks_first
            iter_hunk2 = count_hunks_second
            iter_commit1 = list1
            iter_commit2 = list2

        different_hunk_segment1 = []
        different_hunk_segment2 = []
        for i in range(len(iter_hunk1)):
            each_hunk_list = ''
            if 0 <= i < len(iter_hunk1)-1:
                each_hunk_list =  iter_commit1[iter_hunk1[i]:iter_hunk1[i+1]]        
            if i == len(iter_hunk1)-1:
                each_hunk_list =  iter_commit1[iter_hunk1[i]:]                                      
            different_hunk_segment1.append(each_hunk_list) 
        for i in range(len(iter_hunk2)):
            each_hunk_list=''
            if 0 < i < len(iter_hunk2)-1:
                each_hunk_list =  iter_commit2[iter_hunk2[i]:iter_hunk2[i+1]]        
            if i == len(iter_hunk2)-1:
                each_hunk_list =  iter_commit2[iter_hunk2[i]:]                                      
            different_hunk_segment2.append(each_hunk_list)

        for n, m in enumerate(different_hunk_segment1):          
            for j, k in enumerate(different_hunk_segment2):
                if target_commit == list1:
                    c1 = overlap_percent_check(list1,m,k)
                if target_commit == list2:
                    c1 = overlap_percent_check(m,list2,k)
                if target_commit == list3:
                    c1 = overlap_percent_check(m,k,list3)
                if c1 == True:
                    count_repeat_change += 1
                c1 = False
                
    # 1.4 [>1,>1,>1] - all commits had changed more than one code areas
    if hunk_distribu.count(1) == 0:
        different_hunk_segment1 = []
        different_hunk_segment2 = []
        different_hunk_segment3 = []
        for i in range(len(count_hunks_first)):
            each_hunk_list = ''
            if 0 <= i < len(count_hunks_first)-1:
                each_hunk_list =  list1[count_hunks_first[i]:count_hunks_first[i+1]]        
            if i == len(count_hunks_first)-1:
                each_hunk_list =  list1[count_hunks_first[i]:]                                      
            different_hunk_segment1.append(each_hunk_list)

        for i in range(len(count_hunks_second)):
            each_hunk_list=''
            if 0 <= i < len(count_hunks_second)-1:
                each_hunk_list =  list2[count_hunks_second[i]:count_hunks_second[i+1]]        
            if i == len(count_hunks_second)-1:
                each_hunk_list =  list2[count_hunks_second[i]:]                                      
            different_hunk_segment2.append(each_hunk_list)

        for i in range(len(count_hunks_third)):
            each_hunk_list = ''
            if 0 <= i < len(count_hunks_third)-1:
                # 中间的提交,截取当前下标位置和下一个下标位置之间行
                each_hunk_list =  list3[count_hunks_third[i]:count_hunks_third[i+1]]        
            if i == len(count_hunks_third)-1:
                each_hunk_list =  list3[count_hunks_third[i]:]                                      
            different_hunk_segment3.append(each_hunk_list)

        for n, m in enumerate(different_hunk_segment1):
            for j, k in enumerate(different_hunk_segment2):
                for e, g in enumerate(different_hunk_segment3):
                    c1 = overlap_percent_check(m,k,g)
                    if c1 == True:
                        count_repeat_change += 1
    return count_repeat_change  

# function - comparing three successive commits
def check_repeat_change_in_successive_commit(mylist):
    print('Total number of commit',len(mylist))
    count_combination = 0
    count_repeat_change = 0     
    count_hunks_first = [] # first hunk
    count_hunks_second = [] # second hunk
    count_hunks_third = [] # third hunk    
    first_commit_file_loc_plus = []
    first_commit_file_loc_minus = []    
    second_commit_file_loc_plus = []
    second_commit_file_loc_minus = []    
    third_commit_file_loc_plus = []
    third_commit_file_loc_minus = []
    
    # the total number of commits must larger than three 
    for x in range(len(mylist)-3):
        #using list slicing to jump ahead three elements
        new_list = mylist[x:x+3]
        count_combination += 1
        c = 0     
        count_hunks_first.clear()
        count_hunks_second.clear()
        count_hunks_third.clear()        
        first_commit_file_loc_plus.clear()
        first_commit_file_loc_minus.clear()
        second_commit_file_loc_plus.clear()
        second_commit_file_loc_minus.clear()
        third_commit_file_loc_plus.clear()
        third_commit_file_loc_minus.clear()

        # count the number of hunk for each commit, 
        # store the corresponding index and file name
        for idx, val in enumerate(new_list[0]):
            if val.startswith('@@') == True:
                count_hunks_first.append(idx)
            #+++
            if val.startswith('+++') == True:
                first_commit_file_loc_plus.append(idx)
            #---
            if val.startswith('---') == True:
                first_commit_file_loc_minus.append(idx)                
                
        for idx, val in enumerate(new_list[1]):
            #@@
            if val.startswith('@@') == True:
                count_hunks_second.append(idx)
            #+++
            if val.startswith('+++') == True:
                second_commit_file_loc_plus.append(idx)
            #---
            if val.startswith('---') == True:
                second_commit_file_loc_minus.append(idx)  
                
        for idx, val in enumerate(new_list[2]):
            # @@
            if val.startswith('@@') == True:
                count_hunks_third.append(idx)
            #+++
            if val.startswith('+++') == True:
                third_commit_file_loc_plus.append(idx)
            #---
            if val.startswith('---') == True:
                third_commit_file_loc_minus.append(idx)                        
        # for check file numbers
        file_distribu = [len(first_commit_file_loc_plus),len(second_commit_file_loc_plus),len(third_commit_file_loc_plus)]

        # Case 1 (1,1,1) - all commits changed only one file
        if len(first_commit_file_loc_plus) == 1 and len(second_commit_file_loc_plus) == 1 and len(third_commit_file_loc_plus) == 1:
            f_file = (new_list[0][first_commit_file_loc_plus[0]])
            s_file = (new_list[1][second_commit_file_loc_plus[0]])
            t_file = (new_list[2][third_commit_file_loc_plus[0]])
            if f_file == s_file == t_file:
                # handle different hunks
                c = handle_different_number_of_hunks(new_list[0],new_list[1],new_list[2],count_hunks_first,count_hunks_second,count_hunks_third)                          

        # Case 2（1，1，>1） - two commits both changed only one file, the third commit changed more than one file
        if file_distribu.count(1) == 2:
            iter_commit = []
            iter_file_list = []
            file1 = ''
            file2 = ''
            each_file_list = ''
            new_file_check = ''
            if len(first_commit_file_loc_plus) > 1:
                iter_file_list = first_commit_file_loc_plus
                iter_commit = new_list[0]
                file1 = (new_list[1][second_commit_file_loc_plus[0]])
                file2 = (new_list[2][third_commit_file_loc_plus[0]])
            if len(second_commit_file_loc_plus) > 1:
                iter_file_list = second_commit_file_loc_plus
                iter_commit = new_list[1]
                file1 = (new_list[0][first_commit_file_loc_plus[0]])
                file2 = (new_list[2][third_commit_file_loc_plus[0]])
            if len(third_commit_file_loc_plus) > 1:
                iter_file_list = third_commit_file_loc_plus
                iter_commit = new_list[2]
                file1 = (new_list[0][first_commit_file_loc_plus[0]])
                file2 = (new_list[1][second_commit_file_loc_plus[0]])

            different_file_segment = []
            for i in range(len(iter_file_list)):
                if 0 <= i < len(iter_file_list)-1:
                    each_file_list = iter_commit[iter_file_list[i]:iter_file_list[i+1]-1]        
                if i == len(iter_file_list)-1:
                    each_file_list = iter_commit[iter_file_list[i]:]
                different_file_segment.append(each_file_list)

            for n, each_file_section in enumerate(different_file_segment):
                current_file1 = ''
                for i in range(len(different_file_segment)):
                    count_hunk = 'count'+str(i)
                    locals()['count'+str(i)] = i
                    count_hunk = []
                    for n, m in enumerate(each_file_section):
                        if m.startswith('+++') == True:
                            current_file1 = m
                        if m.startswith('@@') == True:
                            count_hunk.append(n)

                    if file1 == file2 == current_file1:
                        if iter_commit == new_list[0]:
                            # handle different hunks
                            c = handle_different_number_of_hunks(each_file_section,new_list[1],new_list[2],count_hunk,count_hunks_second,count_hunks_third)                          
                        if iter_commit == new_list[1]:
                            c = handle_different_number_of_hunks(new_list[0],each_file_section,new_list[2],count_hunks_first,count_hunk,count_hunks_third)                          
                        if iter_commit == new_list[2]:
                            c = handle_different_number_of_hunks(new_list[0],new_list[1],each_file_section,count_hunks_first,count_hunks_second,count_hunk)                          

        # Casw 3 （1，>1，>1）- one commit changed only one file, the other two commits changed more than one file
        if file_distribu.count(1) == 1:
            target_filenamme = ''
            target_commit = iter_commit1 = iter_commit2 = iter_files1 = iter_files2 = []

            if len(first_commit_file_loc_plus) == 1:
                target_commit = new_list[0]
                target_filenamme = (new_list[0][first_commit_file_loc_plus[0]])
                iter_files1 = second_commit_file_loc_plus
                iter_files2 = third_commit_file_loc_plus
                iter_commit1 = new_list[1]
                iter_commit2 = new_list[2]
            if len(second_commit_file_loc_plus) == 1:
                target_commit = new_list[1]
                target_filenamme = (new_list[1][second_commit_file_loc_plus[0]])
                iter_files1 = first_commit_file_loc_plus
                iter_files2 = third_commit_file_loc_plus
                iter_commit1 = new_list[0]
                iter_commit2 = new_list[2]
            if len(third_commit_file_loc_plus) == 1:
                target_commit = new_list[2]
                target_filenamme = (new_list[2][third_commit_file_loc_plus[0]])
                iter_files1 = first_commit_file_loc_plus
                iter_files2 = second_commit_file_loc_plus
                iter_commit1 = new_list[0]
                iter_commit2 = new_list[1]

            different_file_segment1 = []
            different_file_segment2 = []
            for i in range(len(iter_files1)):
                each_file_list = ''
                if 0 <= i < len(iter_files1)-1:
                    each_file_list =  iter_commit1[iter_files1[i]:iter_files1[i+1]-1]        
                if i == len(iter_files1)-1:
                    each_file_list =  iter_commit1[iter_files1[i]:]                                      
                different_file_segment1.append(each_file_list) 
            for i in range(len(iter_files2)):
                each_file_list = ''
                if 0 <= i < len(iter_files2)-1:
                    each_file_list =  iter_commit2[iter_files2[i]:iter_files2[i+1]-1]        
                if i == len(iter_files2)-1:
                    each_file_list =  iter_commit2[iter_files2[i]:]                                      
                different_file_segment2.append(each_file_list)

            segment1_filename = []
            segment2_filename = []
            segment1_hunk_number = []
            segment2_hunk_number = []
            each_file_section_1 = []
            each_file_section_2 = []
            for n, each_file_section1 in enumerate(different_file_segment1):
                hunk_loc1 = []
                for t, m in enumerate(each_file_section1):
                    if m.startswith('+++') == True:
                        # store current file names
                        segment1_filename.append(m)
                    if m.startswith('@@') == True:
                        # store current file hunk number 
                        hunk_loc1.append(t)
                segment1_hunk_number.append(hunk_loc1)
                each_file_section_1.append(each_file_section1)
                
            for e, each_file_section2 in enumerate(different_file_segment2):
                hunk_loc2 = []
                for k, j in enumerate(each_file_section2):
                    if j.startswith('+++') == True:
                        segment2_filename.append(j)
                for k, j in enumerate(each_file_section2):
                    if j.startswith('@@') == True: 
                        hunk_loc2.append(k)
                # [[1，6],[1]..]
                segment2_hunk_number.append(hunk_loc2)
                each_file_section_2.append(each_file_section2)
            hunk1 = []
            hunk2 = []
            list1 = []
            list2 = []
            for a, file1 in enumerate(segment1_filename): 
                for b, file2 in enumerate(segment2_filename):
                    if file1 == file2 == target_filenamme:
                        list1 = each_file_section_1[a]
                        list2 = each_file_section_2[b]
                        hunk1 = segment1_hunk_number[a]
                        hunk2 = segment2_hunk_number[b]
                    if target_commit == new_list[0]:
                        c = handle_different_number_of_hunks(new_list[0],list1,list2,count_hunks_first,hunk1,hunk2)                          
                    if target_commit == new_list[1]:
                        c = handle_different_number_of_hunks(list1,new_list[1],list2,hunk1,count_hunks_second,hunk2)                          
                    if target_commit == new_list[2]:
                        c = handle_different_number_of_hunks(list1,list2,new_list[2],hunk1,hunk2,count_hunks_third)                          

        # Casw 4 （>1，>1，>1）- all commits had changed more than one file
        if file_distribu.count(1) == 0:
            different_file_segment1 = []
            different_file_segment2 = []
            different_file_segment3 = []
            for i in range(len(first_commit_file_loc_plus)):
                each_file_section = ''
                if 0 <= i < len(first_commit_file_loc_plus)-1:
                    each_file_section =  new_list[0][first_commit_file_loc_plus[i]:first_commit_file_loc_plus[i+1]-1]        
                if i == len(first_commit_file_loc_plus)-1:
                    each_file_section =  new_list[0][first_commit_file_loc_plus[i]:]                                      
                different_file_segment1.append(each_file_section)

            for i in range(len(second_commit_file_loc_plus)):
                each_file_section = ''
                if 0 <= i < len(second_commit_file_loc_plus)-1:
                    each_file_section =  new_list[1][second_commit_file_loc_plus[i]:second_commit_file_loc_plus[i+1]-1]        
                if i == len(second_commit_file_loc_plus)-1:
                    each_file_section =  new_list[1][second_commit_file_loc_plus[i]:]                                      
                different_file_segment2.append(each_file_section)

            for i in range(len(third_commit_file_loc_plus)):
                each_file_section = ''
                if 0 <= i < len(third_commit_file_loc_plus)-1:
                    each_file_section =  new_list[2][third_commit_file_loc_plus[i]:third_commit_file_loc_plus[i+1]-1]        
                if i == len(third_commit_file_loc_plus)-1:
                    each_file_section =  new_list[2][third_commit_file_loc_plus[i]:]                                      
                different_file_segment3.append(each_file_section)
            segment1_filename = []
            segment2_filename = []
            segment3_filename = []
            segment1_hunk_number = []
            segment2_hunk_number = []
            segment3_hunk_number = []
            each_file_section_1 = []
            each_file_section_2 = []
            each_file_section_3 = []

            for n, each_file_section1 in enumerate(different_file_segment1):
                hunk_loc1 = []
                for p, m in enumerate(each_file_section1):
                    if m.startswith('+++') == True:
                        segment1_filename.append(m)
                    if m.startswith('@@') == True:
                        # store current file hunk number 
                        hunk_loc1.append(p)
                segment1_hunk_number.append(hunk_loc1)
                each_file_section_1.append(each_file_section1)

            for j, each_file_section2 in enumerate(different_file_segment2):
                hunk_loc2 = []
                for r, t in enumerate(each_file_section2):
                    if t.startswith('+++') == True:
                        segment2_filename.append(t)
                    if t.startswith('@@') == True:
                        hunk_loc2.append(r)
                segment2_hunk_number.append(hunk_loc2)
                each_file_section_2.append(each_file_section2)

            for e, each_file_section3 in enumerate(different_file_segment3):
                hunk_loc3 = []
                for c, d in enumerate(each_file_section3):
                    if d.startswith('+++') == True:
                        segment3_filename.append(d)
                    if d.startswith('@@') == True:
                        hunk_loc3.append(c)
                segment3_hunk_number.append(hunk_loc3)
                each_file_section_3.append(each_file_section3)
            hunk1 = []
            hunk2 = []
            hunk3 = []
            list1 = []
            list2 = []
            list3 = []
            li = []
            for a, file1 in enumerate(segment1_filename): 
                s = 0
                for b, file2 in enumerate(segment2_filename):
                    for c, file3 in enumerate(segment3_filename):
                        x = 0
                        if file1 == file2 == file3:
                            list1 = each_file_section_1[a]
                            list2 = each_file_section_2[b]
                            list3 = each_file_section_3[c]
                            hunk1 = segment1_hunk_number[a]
                            hunk2 = segment2_hunk_number[b]   
                            hunk3 = segment3_hunk_number[c] 
                            x = handle_different_number_of_hunks(list1,list2,list3,hunk1,hunk2,hunk3)
                            li.append(x)                            
            c = sum(li)
        count_repeat_change = count_repeat_change + c
    print('NumberOfRepeatChangeArea',count_repeat_change)
    
import re  
import Levenshtein
# match Date line
p = "(^Date:)(\s*)(Mon|Thu|Tue|Wed|Fri|Sat|Sun)(.+)" 
# the list storing the diff file name for each project, for example three projects below
diff_list = ['4001','4002','4015']

for gg, each_diff_file in enumerate(diff_list):    
    diffs_file = path_to_diff_files + each_diff_file +'.txt'
    f = open(diffs_file,'r',encoding='UTF-8',errors='ignore')
    diffs_list = f.readlines() 

    # convert txt to [[C1],[C2],[]...]
    store_date_line_index = []
    for index, val in enumerate(diffs_list):
        match_date = re.match(p,val,flags=0) 
        if match_date != None: 
            store_date_line_index.append(index)
    list_after_split = []
    for i in range(len(store_date_line_index)):
        sub_list_for_each_commit='C_'+str(i+1)
        locals()['C_'+str(i+1)]=i
        # the first commit
        if i == 0:
            sub_list_for_each_commit = diffs_list[:store_date_line_index[1]]
        # the last commit
        if i == (len(store_date_line_index)-1):
            sub_list_for_each_commit = diffs_list[store_date_line_index[i]:] 
        if 0 < i < len(store_date_line_index)-1:
            sub_list_for_each_commit =  diffs_list[store_date_line_index[i]:store_date_line_index[i+1]]

        list_after_split.append(sub_list_for_each_commit) 
    print('ProjectID',each_diff_file)
    check_repeat_change_in_successive_commit(list_after_split)